# Tensorflow Keras example with SavedModel model saving
---

<font color='red'> <h3>Tested with TensorFlow 2.2.0</h3></font>

<p>
<h1>Machine Learning on <a href="https://github.com/logicalclocks/hopsworks">Hopsworks
</a></h1> 
</p>

![hops.png](../../images/hops.png)

## The `hops` python module

`hops` is a helper library for Hops that facilitates development by hiding the complexity of running applications and iteracting with services.

Have a feature request or encountered an issue? Please let us know on <a href="https://github.com/logicalclocks/hops-util-py">github</a>.

### Using the `experiment` module

To be able to run your Machine Learning code in Hopsworks, the code for the whole program needs to be provided and put inside a wrapper function. Everything, from importing libraries to reading data and defining the model and running the program needs to be put inside a wrapper function.

The `experiment` module provides an api to Python programs such as TensorFlow, Keras and PyTorch on a Hopsworks on any number of machines and GPUs.

An Experiment could be a single Python program, which we refer to as an **Experiment**. 

Grid search or genetic hyperparameter optimization such as differential evolution which runs several Experiments in parallel, which we refer to as **Parallel Experiment**. 

ParameterServerStrategy, CollectiveAllReduceStrategy and MultiworkerMirroredStrategy making multi-machine/multi-gpu training as simple as invoking a function for orchestration. This mode is referred to as **Distributed Training**.

### Using the `tensorboard` module
The `tensorboard` module allow us to get the log directory for summaries and checkpoints to be written to the TensorBoard we will see in a bit. The only function that we currently need to call is `tensorboard.logdir()`, which returns the path to the TensorBoard log directory. Furthermore, the content of this directory will be put in as a Dataset in your project's Experiments folder.

The directory could in practice be used to store other data that should be accessible after the experiment is finished.
```python
# Use this module to get the TensorBoard logdir
from hops import tensorboard
tensorboard_logdir = tensorboard.logdir()
```

### Using the `hdfs` module
The `hdfs` module provides a method to get the path in HopsFS where your data is stored, namely by calling `hdfs.project_path()`. The path resolves to the root path for your project, which is the view that you see when you click `Data Sets` in HopsWorks. To point where your actual data resides in the project you to append the full path from there to your Dataset. For example if you create a mnist folder in your Resources Dataset, the path to the mnist data would be `hdfs.project_path() + 'Resources/mnist'`

```python
# Use this module to get the path to your project in HopsFS, then append the path to your Dataset in your project
from hops import hdfs
project_path = hdfs.project_path()
```

```python
# Downloading the mnist dataset to the current working directory
from hops import hdfs
mnist_hdfs_path = hdfs.project_path() + "Resources/mnist"
local_mnist_path = hdfs.copy_to_local(mnist_hdfs_path)
```

### Documentation
See the following links to learn more about running experiments in Hopsworks

- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/experiment.html">Learn more about experiments</a>
<br>
- <a href="https://hopsworks.readthedocs.io/en/latest/hopsml/hopsML.html">Building End-To-End pipelines</a>
<br>
- Give us a star, create an issue or a feature request on  <a href="https://github.com/logicalclocks/hopsworks">Hopsworks github</a>

### Managing experiments
Experiments service provides a unified view of all the experiments run using the `experiment` module.
<br>
As demonstrated in the gif it provides general information about the experiment and the resulting metric. Experiments can be visualized meanwhile or after training in a TensorBoard.
<br>
<br>
![Image7-Monitor.png](../../images/experiments.gif)

In [15]:
def keras_mnist():
    
    import os
    import sys
    import uuid
    import random
    
    import numpy as np
    
    from tensorflow import keras
    import tensorflow as tf
    from tensorflow.keras.datasets import mnist
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Dropout, Flatten
    from tensorflow.keras.layers import Conv2D, MaxPooling2D
    from tensorflow.keras.callbacks import TensorBoard
    from tensorflow.keras import backend as K

    import math
    from hops import tensorboard

    from hops import model as hops_model
    from hops import hdfs
    
    batch_size = 8
    num_classes = 10
    epochs = 3
    kernel = 4
    pool = 4
    dropout = 0.5
#################################################################################
#     # Input image dimensions
#     img_rows, img_cols = 28, 28

#     # The data, shuffled and split between train and test sets
#     (x_train, y_train), (x_test, y_test) = mnist.load_data()

#     if K.image_data_format() == 'channels_first':
#         x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
#         x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
#         input_shape = (1, img_rows, img_cols)
#     else:
#         x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
#         x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
#         input_shape = (img_rows, img_cols, 1)

#     x_train = x_train.astype('float32')
#     x_test = x_test.astype('float32')
#     x_train /= 255
#     x_test /= 255
#     print('x_train shape:', x_train.shape)
#     print(x_train.shape[0], 'train samples')
#     print(x_test.shape[0], 'test samples')

#     # Convert class vectors to binary class matrices
#     y_train = keras.utils.to_categorical(y_train, num_classes)
#     y_test = keras.utils.to_categorical(y_test, num_classes)

#     model = Sequential()
#     model.add(Conv2D(32, kernel_size=(kernel, kernel),
#                         activation='relu',
#                          input_shape=input_shape))
#     model.add(Conv2D(64, (kernel, kernel), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(pool, pool)))
#     model.add(Dropout(dropout))
#     model.add(Flatten())
#      model.add(Dense(128, activation='relu'))

#     model.add(Dropout(dropout))
#     model.add(Dense(num_classes, activation='softmax'))

#     opt = keras.optimizers.Adadelta(1.0)

#     model.compile(loss=keras.losses.categorical_crossentropy,
#                       optimizer=opt,
#                       metrics=['accuracy'])


#     tb_callback = TensorBoard(log_dir=tensorboard.logdir(), histogram_freq=0,
#                              write_graph=True, write_images=True)
#     callbacks = [tb_callback]

#     model.fit(x_train, y_train,
#              batch_size=batch_size,
#              callbacks=callbacks,
#              epochs=epochs,
#              verbose=1,
#              validation_data=(x_test, y_test))
#     score = model.evaluate(x_test, y_test, verbose=0)


    # Define input function
    def input_fn():
      x = np.random.random((1024, 10))
      y = np.random.randint(2, size=(1024, 1))
      x = tf.cast(x, tf.float32)
      dataset = tf.data.Dataset.from_tensor_slices((x, y))
      dataset = dataset.repeat(500)
      dataset = dataset.batch(batch_size)
      return dataset

    # Define a Keras Model.
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(16, activation='relu', input_shape=(10,)))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compile the model.
    optimizer = tf.keras.optimizers.Adam(0.001) 
    # Compile the model.
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy']
                 )
        
    callbacks = [
        tf.keras.callbacks.TensorBoard(log_dir=tensorboard.logdir()),
        tf.keras.callbacks.ModelCheckpoint(filepath=tensorboard.logdir()),
    ]
    model.fit(input_fn(), 
        verbose=0,
        epochs=3, 
        steps_per_epoch=5,
        validation_data=input_fn(),
        callbacks=callbacks
    )
    
    score = model.evaluate(input_fn())

    # Export model
    # WARNING(break-tutorial-inline-code): The following code snippet is
    # in-lined in tutorials, please update tutorial documents accordingly
    # whenever code changes.

    export_path = os.getcwd() + '/model-' + str(uuid.uuid4())
    print('Exporting trained model to: {}'.format(export_path))
    
#     tf.keras.models.save_model(
#         model,
#         export_path,
#         overwrite=True,
#         include_optimizer=True,
#         save_format=None,
#         signatures=None,
#         options=None
#     )
    
    tf.saved_model.save(model, export_path)

    print('Done exporting!')
    
    metrics = metrics={'accuracy': score[1]}
    
    hops_model.export(export_path, "fakedataB", metrics=metrics)    
    
    return metrics

In [ ]:
from hops import experiment
from hops import hdfs

experiment.launch(keras_mnist, name='keras fake data', local_logdir=True, metric_key='accuracy')

# Check Model Repository for best model based on accuracy

![Image7-Monitor.png](../../images/models.gif)

# Query Model Repository for best mnist Model

In [ ]:
from hops import model
from hops.model import Metric
MODEL_NAME="fakedataB"
EVALUATION_METRIC="accuracy"

In [2]:
best_model = model.get_best_model(MODEL_NAME, EVALUATION_METRIC, Metric.MAX)

In [3]:
print('Model name: ' + best_model['name'])
print('Model version: ' + str(best_model['version']))
print(best_model['metrics'])

Model name: fakedata
Model version: 1
{'accuracy': '0.48828125'}

# Create Model Serving of Exported Model

In [4]:
from hops import serving

In [5]:
# Create serving
model_path="/Models/" + best_model['name']
response = serving.create_or_update(model_path, MODEL_NAME, serving_type="TENSORFLOW", 
                                 model_version=best_model['version'])

Creating a serving for model fakedata ...
Serving for model fakedata successfully created

In [6]:
# List all available servings in the project
for s in serving.get_all():
    print(s.name)

fakedata
mnist

In [7]:
# Get serving status
serving.get_status(MODEL_NAME)

'Running'

# Check Model Serving for active servings

![Image7-Monitor.png](../../images/servings.gif)

# Start Model Serving Server

In [8]:
if serving.get_status(MODEL_NAME) == 'Stopped':
    serving.start(MODEL_NAME)

In [9]:
import time
while serving.get_status(MODEL_NAME) != "Running":
    time.sleep(5) # Let the serving startup correctly
time.sleep(5)

# Send Prediction Requests to the Served Model using Hopsworks REST API

In [10]:
import numpy as np
TOPIC_NAME = serving.get_kafka_topic(MODEL_NAME)
NUM_FEATURES=1024

In [14]:
for i in range(20):
    data = {
                "signature_name": "serving_default", "instances": [np.random.rand(NUM_FEATURES).tolist()]
            }
    response = serving.make_inference_request(MODEL_NAME, data)
    print(response)

An error was encountered:
Could not create or update serving (url: /hopsworks-api/api/project/128/inference/models/fakedata:predict), server response: 
 HTTP code: 400, HTTP reason: Bad Request, error code: 250008, error msg: Serving instance bad request error, user msg: { "error": "Matrix size-incompatible: In[0]: [1,1024], In[1]: [10,16]\n\t [[{{node sequential/dense/Relu}}]]" }
Traceback (most recent call last):
  File "/srv/hops/anaconda/envs/theenv/lib/python3.6/site-packages/hops/serving.py", line 527, in make_inference_request
    return _make_inference_request_rest(serving_name, data, verb)
  File "/srv/hops/anaconda/envs/theenv/lib/python3.6/site-packages/hops/serving.py", line 562, in _make_inference_request_rest
    error_code, error_msg, user_msg))
hops.exceptions.RestAPIError: Could not create or update serving (url: /hopsworks-api/api/project/128/inference/models/fakedata:predict), server response: 
 HTTP code: 400, HTTP reason: Bad Request, error code: 250008, error msg: